# Использование библиотеки SKLEARN для обученя модели 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import download
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn  # Проверить необходимость

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier  # SVM модель
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from gensim.models import Word2Vec
from collections import defaultdict, Counter
import openpyxl

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aleksei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aleksei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Aleksei\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Aleksei\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aleksei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Подгружаем датасет для работы с ним

In [3]:
summaries = None
chunksize = 10 ** 5
with pd.read_csv("fz.csv", chunksize=chunksize, on_bad_lines="skip") as reader:
    for chunk in reader:
#        chunk.columns = ["id1", "Реестровый номер контракта", "id2", "юр лицо", "юр лицо", "Неизвестно", 
#              "id4", "юр лицо", "регион", "id5", "Результат", "Услуга", "Откуда деньги", 
#              "Дата 1", "Дата 2", "Дата 3", "Дата 4", "Дата 5", "Дата 6", "Сумма 1", "Сумма 2", 
#              "Сумма 3", "Условия 1", "Условия 2", "Услуга 1", "Услуга 2", "ОКПД-2"]
        chunk.columns = ["Номер", "Реестровый номер контракта", "Код закупки", "ИНН поставщика", "Наименование поставщика", 
           "Регион поставщика", "ИНН заказчика", "Наименование заказчика", "Регион заказчика", "Код КБК", "Исполнение",
           "Предмет закупки", "Наименование бюджета", "Дата заключения", "Дата начала", "Дата окончания",
           "Дата обновления", "Дата обновления_1", "null_1", "null_2", "Цена контракта", "Стоимость обязательств", 
           "НДС", "Изменение условий контракта", "null_3", "Описание ОКПД", "Код ОКПД"]
        chunk["Код ОКПД"] = chunk["Код ОКПД"].astype(str)
        summary = chunk.groupby(['Описание ОКПД','Код ОКПД']).size().reset_index().rename(columns={0:'count'})
        if summaries is None:
            summaries = summary
        else:
            summaries = pd.concat([summaries, summary])
        
        break

C:\Users\Aleksei\AppData\Local\Temp\ipykernel_3660\183269662.py:4: DtypeWarning: Columns (6,21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


In [4]:
chunk.shape

(100000, 27)

In [5]:
chunk["Предмет закупки"]

0                                          <НЕ ОПРЕДЕЛЕНО>
1                                          <НЕ ОПРЕДЕЛЕНО>
2        На поставку специальных устройств для чтения «...
3        На поставку голосообразующих аппаратов для обе...
4                                          <НЕ ОПРЕДЕЛЕНО>
                               ...                        
99995                                      <НЕ ОПРЕДЕЛЕНО>
99996      ПОСТАВКА ЛЕКАРСТВЕННОГО ПРЕПАРАТА (БИКАЛУТАМИД)
99997                                      <НЕ ОПРЕДЕЛЕНО>
99998                                      <НЕ ОПРЕДЕЛЕНО>
99999                                      <НЕ ОПРЕДЕЛЕНО>
Name: Предмет закупки, Length: 100000, dtype: object

In [6]:
chunk["Код ОКПД"] = chunk["Код ОКПД"].astype(str)
chunk = chunk[chunk['Код ОКПД'].map(lambda x: str.isnumeric(x[0]))]
chunk = chunk[chunk["Предмет закупки"] != "<НЕ ОПРЕДЕЛЕНО>"]

In [7]:
chunk['Код ОКПД'].shape

(72159,)

Убираем ненужные записи из датасета

In [8]:
orig_text = chunk[["Предмет закупки", "Код ОКПД"]]
orig_text.columns = ["data", "Код ОКПД"]

In [9]:
orig_text

,data,Код ОКПД
2,На поставку специальных устройств для чтения «...,26.4
3,На поставку голосообразующих аппаратов для обе...,26.4
5,на поставку слуховых аппаратов (включая настро...,26.6
6,На выполнение работ по изготовлению вкладышей ...,26.6
7,На поставку специальных средств при нарушениях...,32.5
...,...,...
99984,ПОСТАВКА ЛЕКАРСТВЕННОГО ПРЕПАРАТА (ПРАМИПЕКСОЛ),21.2
99985,ПОСТАВКА ЛЕКАРСТВЕННОГО ПРЕПАРАТА (БОЗЕНТАН),21.2
99986,ПОСТАВКА ЛЕКАРСТВЕННОГО ПРЕПАРАТА (МЕРОПЕНЕМ),21.2
99987,ПОСТАВКА ЛЕКАРСТВЕННОГО ПРЕПАРАТА (ОНДАНСЕТРОН),21.2


In [10]:
# orig_text = orig_text.head(10000)  # Немного уберём данные
orig_text = orig_text.dropna()
orig_text["data"] = orig_text["data"].str.replace('\W', ' ')  # Удалим спецсимволы
orig_text["data"] = orig_text["data"].str.replace('\s+[a-zA-Z]\s+', ' ')  # Удалим одиночные литеры
orig_text["data"] = orig_text["data"].str.replace('\^[a-zA-Z]\s+', ' ')  # Удалим одиночные литеры в начале
orig_text["data"] = orig_text["data"].str.replace('\s+', ' ')  # Удалим множественные пробелы
orig_text["data"] = orig_text["data"].str.replace('\d+', '')  # Удалим цифры

C:\Users\Aleksei\AppData\Local\Temp\ipykernel_3660\2633631860.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  orig_text["data"] = orig_text["data"].str.replace('\W', ' ')  # Удалим спецсимволы
C:\Users\Aleksei\AppData\Local\Temp\ipykernel_3660\2633631860.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  orig_text["data"] = orig_text["data"].str.replace('\s+[a-zA-Z]\s+', ' ')  # Удалим одиночные литеры
C:\Users\Aleksei\AppData\Local\Temp\ipykernel_3660\2633631860.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  orig_text["data"] = orig_text["data"].str.replace('\^[a-zA-Z]\s+', ' ')  # Удалим одиночные литеры в начале
C:\Users\Aleksei\AppData\Local\Temp\ipykernel_3660\2633631860.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  orig_text["data"] = orig_text["data"].str.replace('\s

Токенизация.
Удаляем ненужные слова

In [11]:
orig_text["data"] = [word_tokenize(entry.lower()) for entry in orig_text["data"]]
orig_text["data"]

2        [на, поставку, специальных, устройств, для, чт...
3        [на, поставку, голосообразующих, аппаратов, дл...
5        [на, поставку, слуховых, аппаратов, включая, н...
6        [на, выполнение, работ, по, изготовлению, вкла...
7        [на, поставку, специальных, средств, при, нару...
                               ...                        
99984    [поставка, лекарственного, препарата, прамипек...
99985      [поставка, лекарственного, препарата, бозентан]
99986     [поставка, лекарственного, препарата, меропенем]
99987    [поставка, лекарственного, препарата, ондансет...
99996    [поставка, лекарственного, препарата, бикалута...
Name: data, Length: 72159, dtype: object

Лемматизация. Получение массива нужных слов.

In [12]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

stemmer = nltk.stem.snowball.RussianStemmer()
word_lemmatized = WordNetLemmatizer()

for index, entry in zip(orig_text["data"].index, orig_text["data"]):
    # Declaring Empty List to store the words that follow the rules for this step
    final_words = []
    # Initializing WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words("russian") and word.isalpha():
            word_final = word_lemmatized.lemmatize(word, tag_map[tag[0]])
            word_final = stemmer.stem(word_final)
            final_words.append(word_final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    orig_text.loc[index, "data_final"] = str(final_words)
    
orig_text["data_final"]

2        ['поставк', 'специальн', 'устройств', 'чтен', ...
3        ['поставк', 'голосообраз', 'аппарат', 'обеспеч...
5        ['поставк', 'слухов', 'аппарат', 'включ', 'нас...
6        ['выполнен', 'работ', 'изготовлен', 'вкладыш',...
7        ['поставк', 'специальн', 'средств', 'нарушен',...
                               ...                        
99984    ['поставк', 'лекарствен', 'препарат', 'прамипе...
99985     ['поставк', 'лекарствен', 'препарат', 'бозента']
99986     ['поставк', 'лекарствен', 'препарат', 'меропен']
99987    ['поставк', 'лекарствен', 'препарат', 'ондансе...
99996    ['поставк', 'лекарствен', 'препарат', 'бикалут...
Name: data_final, Length: 72159, dtype: object

In [13]:
orig_text

,data,Код ОКПД,data_final
2,"[на, поставку, специальных, устройств, для, чт...",26.4,"['поставк', 'специальн', 'устройств', 'чтен', ..."
3,"[на, поставку, голосообразующих, аппаратов, дл...",26.4,"['поставк', 'голосообраз', 'аппарат', 'обеспеч..."
5,"[на, поставку, слуховых, аппаратов, включая, н...",26.6,"['поставк', 'слухов', 'аппарат', 'включ', 'нас..."
6,"[на, выполнение, работ, по, изготовлению, вкла...",26.6,"['выполнен', 'работ', 'изготовлен', 'вкладыш',..."
7,"[на, поставку, специальных, средств, при, нару...",32.5,"['поставк', 'специальн', 'средств', 'нарушен',..."
...,...,...,...
99984,"[поставка, лекарственного, препарата, прамипек...",21.2,"['поставк', 'лекарствен', 'препарат', 'прамипе..."
99985,"[поставка, лекарственного, препарата, бозентан]",21.2,"['поставк', 'лекарствен', 'препарат', 'бозента']"
99986,"[поставка, лекарственного, препарата, меропенем]",21.2,"['поставк', 'лекарствен', 'препарат', 'меропен']"
99987,"[поставка, лекарственного, препарата, ондансет...",21.2,"['поставк', 'лекарствен', 'препарат', 'ондансе..."


In [14]:
orig_text = orig_text.dropna()
data, y, data_final = orig_text["data"], orig_text["Код ОКПД"], orig_text["data_final"]

In [15]:
data

2        [на, поставку, специальных, устройств, для, чт...
3        [на, поставку, голосообразующих, аппаратов, дл...
5        [на, поставку, слуховых, аппаратов, включая, н...
6        [на, выполнение, работ, по, изготовлению, вкла...
7        [на, поставку, специальных, средств, при, нару...
                               ...                        
99984    [поставка, лекарственного, препарата, прамипек...
99985      [поставка, лекарственного, препарата, бозентан]
99986     [поставка, лекарственного, препарата, меропенем]
99987    [поставка, лекарственного, препарата, ондансет...
99996    [поставка, лекарственного, препарата, бикалута...
Name: data, Length: 72159, dtype: object

In [16]:
data_final

2        ['поставк', 'специальн', 'устройств', 'чтен', ...
3        ['поставк', 'голосообраз', 'аппарат', 'обеспеч...
5        ['поставк', 'слухов', 'аппарат', 'включ', 'нас...
6        ['выполнен', 'работ', 'изготовлен', 'вкладыш',...
7        ['поставк', 'специальн', 'средств', 'нарушен',...
                               ...                        
99984    ['поставк', 'лекарствен', 'препарат', 'прамипе...
99985     ['поставк', 'лекарствен', 'препарат', 'бозента']
99986     ['поставк', 'лекарствен', 'препарат', 'меропен']
99987    ['поставк', 'лекарствен', 'препарат', 'ондансе...
99996    ['поставк', 'лекарствен', 'препарат', 'бикалут...
Name: data_final, Length: 72159, dtype: object

Методы векторизации

Мешок слов

In [17]:
bagwords = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('russian')) 
bagwords_X = bagwords.fit_transform(data_final).toarray()

bagwords_X_train, bagwords_X_test, bagwords_y_train, bagwords_y_test = train_test_split(bagwords_X,
                                                                                        y,
                                                                                        test_size=0.2,
                                                                                        random_state=0)

naive_bayes = MultinomialNB()
bagwords_nb = naive_bayes.fit(bagwords_X_train, bagwords_y_train)
bagwords_nb_predictions = bagwords_nb.predict(bagwords_X_test)

bagwords_nb_f1 = f1_score(bagwords_y_test, bagwords_nb_predictions, average='weighted')
bagwords_nb_precision = precision_score(bagwords_y_test, bagwords_nb_predictions, average='weighted')
print(f'F-мера модели "Мешок слов": {round(bagwords_nb_f1, 3)}, точность: {round(bagwords_nb_precision, 3)}')

F-мера модели "Мешок слов": 0.644, точность: 0.664


C:\Users\Aleksei\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TF-IDF мера для оценки важности слова

In [18]:
tfidf = TfidfVectorizer(max_features=1700, min_df=5, max_df=0.7, stop_words=stopwords.words('russian'))
tfidf_X = tfidf.fit_transform(data_final).toarray()
tfidf_X_train, tfidf_X_test, tfidf_y_train, tfidf_y_test = train_test_split(tfidf_X,
                                                                            y,
                                                                            test_size=0.2,
                                                                            random_state=0)
tfidf_nb = naive_bayes.fit(tfidf_X_train, tfidf_y_train)
tfidf_nb_predictions = tfidf_nb.predict(tfidf_X_test)
tfidf_nb_f1 = f1_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
tfidf_nb_precision = precision_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
print(f'F-мера модели TF-IDF: {round(tfidf_nb_f1, 3)}, точность: {round(tfidf_nb_precision, 3)}')

F-мера модели TF-IDF: 0.623, точность: 0.66


C:\Users\Aleksei\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TF-IDF мера для оценки важности слова

In [19]:
model = DecisionTreeClassifier(max_depth=20, criterion='gini')
tfidf_nb = model.fit(tfidf_X_train, tfidf_y_train)
tfidf_nb_predictions = model.predict(tfidf_X_test)
tfidf_nb_f1 = f1_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
tfidf_nb_precision = precision_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
print(f'F-мера модели TF-IDF: {round(tfidf_nb_f1, 3)}, точность: {round(tfidf_nb_precision, 3)}')

F-мера модели TF-IDF: 0.441, точность: 0.621


C:\Users\Aleksei\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TF-IDF мера для оценки важности слова

In [20]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=1, max_iter=10)
tfidf_nb = model.fit(tfidf_X_train, tfidf_y_train)
tfidf_nb_predictions = model.predict(tfidf_X_test)
tfidf_nb_f1 = f1_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
tfidf_nb_precision = precision_score(tfidf_y_test, tfidf_nb_predictions, average='weighted')
print(f'F-мера модели TF-IDF: {round(tfidf_nb_f1, 3)}, точность: {round(tfidf_nb_precision, 3)}')

F-мера модели TF-IDF: 0.701, точность: 0.711


C:\Users\Aleksei\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Aleksei\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
